In [1]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

# Connect to an in-memory database
con = duckdb.connect()

# --- Your setup code (which is correct) ---
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("""
SET s3_endpoint='122.176.133.73:9000';
SET s3_use_ssl=false;
SET s3_access_key_id='minioadmin';
SET s3_secret_access_key='minioadmin';
""")
con.execute("SET s3_url_style='path';") 
con.execute("PRAGMA threads=8;")
con.execute("PRAGMA enable_object_cache;")

# --- THE MISSING STEP: Querying a file from S3 ---
# Replace with your actual S3 bucket and file path
# This can be a .parquet, .csv, or .json file
s3_file_path = 's3://duckdata/ORATS/Options/**/*.parquet'

try:
    # Execute a query on the S3 file and fetch the result as a Pandas DataFrame
    print(f"Querying data from: {s3_file_path}")
    df = con.execute(f"""
        SELECT *
        FROM '{s3_file_path}'
        LIMIT 2;
    """).df()

    print("\n✅ Query successful! First 5 rows of data:")
    print(df.head())

except Exception as e:
    print(f"\n❌ An error occurred during the query: {e}")

# finally:
#     # It's good practice to close the connection
#     con.close()

Querying data from: s3://duckdata/ORATS/Options/**/*.parquet

✅ Query successful! First 5 rows of data:
      expiry  strike  dte  optionType                  ts     low    high  \
0 2024-01-26   421.0   25          -1 2024-01-02 17:13:00  17.180  17.180   
1 2024-01-12   414.0   11          -1 2024-01-02 17:13:00  10.415  10.415   

     open   close  volume  ...  askSize  bidPrice  askPrice     bidIv  \
0  17.180  17.180     0.0  ...     50.0     17.12     17.24  0.137857   
1  10.415  10.415     0.0  ...    151.0     10.34     10.49  0.142671   

      askIv        iv    oi  stockPrice        day ticker  
0  0.145115  0.141486  15.0       404.0 2024-01-02    QQQ  
1  0.151996  0.147333  54.0       404.0 2024-01-02    QQQ  

[2 rows x 21 columns]


In [ ]:
try:
    print("🔭 Finding all .parquet files using the official read_parquet method...")
    print("(This will be slow due to the massive size of the dataset)")

    # Define the recursive path
    s3_path = 's3://duckdata/ORATS/Options/**/*.parquet'

    # This query uses read_parquet with filename=true to get a list of all files.
    # It is the correct and documented way to handle recursive file discovery.
    all_files_df = con.execute(f"""
        SELECT DISTINCT filename
        FROM read_parquet('{s3_path}', filename=true);
    """).df()

    total_files = len(all_files_df)

    print(f"\n✅ Query complete! Found a total of {total_files:,} files.")
    print("\nHere are the first 10 files found:")

    # The column name from this function is 'filename'
    for f in all_files_df['filename'].head(10).tolist():
        print(f)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

In [ ]:

# --- 2. Define the ticker and date you want ---
target_ticker = 'SPY'
target_day = '2024-07-29'
target_expiry = '2024-08-16'

try:
    print(f"🚀 Running optimized query for {target_ticker} on {target_day}...")

    # This query uses hive_partitioning=true to leverage the folder structure
    data_df = con.execute(f"""
        SELECT *
        FROM read_parquet(
            's3://duckdata/ORATS/Options/**/*.parquet',
            hive_partitioning=true
        )
        WHERE ticker = '{target_ticker}'
          AND day = DATE '{target_day}'
          AND expiry = DATE '{target_expiry}';
    """).df()

    print(f"\n✅ Query successful! Found {len(data_df)} rows of data.")
    print(data_df.head())

except Exception as e:
    print(f"\n❌ An error occurred: {e}")



In [ ]:
import duckdb



# --- 2. Define the ticker and date you want ---
target_ticker = 'QQQ'
target_day = '2024-07-29'

# --- 3. Build a DIRECT path without the recursive '**' glob ---
s3_path = f"s3://duckdata/ORATS/Options/ticker={target_ticker}/day={target_day}/*.parquet"

try:
    print(f"🚀 Running hyper-optimized query with direct path:\n{s3_path}")

    # This query is now as fast as possible because there's no discovery phase.
    data_df = con.execute(f"""
        SELECT
            ts, strike, expiry, bidPrice, stockPrice
        FROM read_parquet('{s3_path}')
        ; -- Limiting rows for a quick test
    """).df()

    print(f"\n✅ Query successful! Found {len(data_df)} rows of data.")
    print(data_df.head())

except Exception as e:
    print(f"\n❌ An error occurred: {e}")


In [ ]:

# --- 2. Define the ticker and date ---
target_ticker = 'SPY'
target_day = '2025-01-02'

# --- 3. Build the direct path for the query ---
s3_path = f"s3://duckdata/ORATS/Options/ticker={target_ticker}/day={target_day}/*.parquet"

try:
    print("🚀 Running query to find near-the-money options...")

    data_df = con.execute(f"""
        SELECT
            ts, strike, expiry, bidPrice, stockPrice
        FROM read_parquet('{s3_path}')
        WHERE
            -- This is the new filter for the strike price
            strike > stockPrice * 0.92 AND strike < stockPrice * 1.04;

    """).df()

    print(f"\n✅ Query successful! Filtered down to {len(data_df)} rows.")
    print(data_df.head())

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

In [ ]:

data_df['ts_eastern'] = data_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York')

In [ ]:
data_df['ts'].unique()

In [ ]:
# --- The One-Liner ---
data_df['ts_eastern'] = pd.to_datetime(data_df['ts']).dt.tz_localize('UTC').dt.tz_convert('America/New_York')


In [ ]:
data_df.sort_index()

In [ ]:
atm = data_df[(data_df['strike'] > data_df["stockPrice"]*0.999) & (data_df['strike'] < data_df["stockPrice"]*1.001)]

In [ ]:
spy_df

In [2]:

spy_df = pd.read_parquet(r"C:\Data\Aggregate_Data\Us_etfs\SPY_15min_2019_2025.parquet")
rdf = pd.read_csv(r"C:\Repos\US_strategies\exposure_series.csv")
rdf['Unnamed: 0'] = pd.to_datetime(rdf["Unnamed: 0"])
rdf.set_index('Unnamed: 0' , inplace=True)
rdf = rdf[rdf.index.year>=2024]
rdf = rdf.between_time('9:30', '15:45')

import numpy as np
# Handle potential NaN values by treating them as 0 for comparison
rdf['net_exposure_filled'] = rdf['net_exposure'].fillna(0)

# 2. Get the exposure from the previous row
rdf['prev_exposure'] = rdf['net_exposure_filled'].shift(1).fillna(0)

# 3. Apply your conditional logic using np.select
conditions = [
    rdf['net_exposure_filled'] > rdf['prev_exposure']+10000,  # Any increase
    rdf['net_exposure_filled'] < rdf['prev_exposure'] - 10000   # Any decrease
]

# Define the outcomes for each condition
outcomes = [1, -1]

# The default value if neither condition is met (i.e., they are equal) is 0
rdf['change_posn'] = np.select(conditions, outcomes, default=0)

rdf['change_in_exposure'] = rdf['net_exposure_filled'] - rdf['prev_exposure']
rdf = rdf.join(spy_df)

In [3]:
rdf


,net_exposure,net_exposure_filled,prev_exposure,change_posn,change_in_exposure,SPY_open,SPY_close,SPY_high,SPY_low,SPY_volume
Unnamed: 0,,,,,,,,,,
2024-01-02 09:30:00,0.000000,0.000000,0.000000,0,0.000000,472.16,471.81,472.80,471.71,5562913.0
2024-01-02 09:45:00,0.000000,0.000000,0.000000,0,0.000000,471.80,472.27,472.28,471.36,4317417.0
2024-01-02 10:00:00,0.000000,0.000000,0.000000,0,0.000000,472.28,472.12,472.71,471.98,2581394.0
2024-01-02 10:15:00,0.000000,0.000000,0.000000,0,0.000000,472.12,471.76,472.42,471.59,2472668.0
2024-01-02 10:30:00,0.000000,0.000000,0.000000,0,0.000000,471.77,471.86,471.96,471.30,2342245.0
...,...,...,...,...,...,...,...,...,...,...
2024-09-06 14:45:00,95974.198063,95974.198063,95821.864262,0,152.333801,540.31,540.45,540.65,539.71,684297.0
2024-09-06 15:00:00,96431.218838,96431.218838,95974.198063,0,457.020775,540.45,541.10,541.43,540.40,962149.0
2024-09-06 15:15:00,96278.875351,96278.875351,96431.218838,0,-152.343487,541.11,541.33,541.88,540.86,1261053.0


In [ ]:
rdf.to_csv(r"test.csv")

In [30]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal

# --- (Your setup code) ---

open_positions = []
trade_log = []
option_multiplier = 100
hedge_ratio = 0.50

# --- 1. Get the NYSE market calendar ---
nyse = mcal.get_calendar('NYSE')
print("🚀 Starting backtest with dynamic quantity...")

# Loop through each row of your exposure data
for timestamp, row in rdf.iterrows():
    marker = row['change_posn']
    current_exposure = row['net_exposure']
    change_in_exposure = row['change_in_exposure']
    
    
    positions_to_keep = []
    if open_positions:
        for position in open_positions:
            expiry_date = pd.to_datetime(position['expiry']).date()
            if timestamp.date() >= expiry_date:
                print(f"   Expiring position with strike {position['strike']} on {timestamp.date()}")
                exit_price = 0.01 # Simulate fetching a closing price
                
                try:
                    # --- DYNAMICALLY FETCH CLOSING PRICE ---
                    # 1. Define the START of the closing window (15:58) and convert to UTC
                    close_window_start_ny = pd.to_datetime(f"{expiry_date} 15:57:00").tz_localize('America/New_York')
                    close_window_end_ny = pd.to_datetime(f"{expiry_date} 15:59:00").tz_localize('America/New_York')
                    close_window_start_utc_str = close_window_start_ny.tz_convert('UTC').strftime('%Y-%m-%d %H:%M:%S')
                    close_window_end_utc_str = close_window_end_ny.tz_convert('UTC').strftime('%Y-%m-%d %H:%M:%S')
                    closing_price_query = f"""
                    SELECT *
                    FROM read_parquet('s3://duckdata/ORATS/Options/ticker=SPY/day={expiry_date.strftime('%Y-%m-%d')}/*.parquet')
                    WHERE 
                        ts >= '{close_window_start_utc_str}'  AND
                        ts <= '{close_window_end_utc_str}'AND
                        optionType = 1 AND
                        strike = {position['strike']}  AND
                        expiry = '{position['expiry']}'  ; """      

                    price_data = con.execute(closing_price_query).df()
                    #print(f"Price data is {price_data}")
                    if not price_data.empty:
                        # 3. Calculate the AVERAGE of the bid prices found
                        exit_price = price_data['askPrice'].mean()
                        print(f"   Found {len(price_data)} prices in closing window. Avg exit price: ${exit_price:.2f}")
                        
                    else:
                        print(f"   No closing price found for strike {position['strike']} on {expiry_date}. Assuming $0.05.")

                except Exception as e:
                    print(f"   Error fetching closing price for strike {position['strike']}: {e}. Assuming $0.00.")

                
                pnl = (position['entry_price'] - exit_price) * position['quantity'] * option_multiplier
                print(f"Pnl of hedge was {pnl}")
                position.update({'exit_ts': timestamp, 'exit_price': exit_price, 'pnl': pnl, 'exit_reason': 'Expired'})
                trade_log.append(position)
            else:
                positions_to_keep.append(position)
        open_positions = positions_to_keep

    # --- 2. HANDLE SIGNALS (SCALE OUT or SCALE IN) ---
    if marker == -1:
        if not open_positions:
            print(f"  ⚠️ {timestamp.date()}: Received SCALE_OUT signal, but no open positions.")
            continue


    # --- ENTRY LOGIC with Quantity Calculation ---
    if marker == 1 :
        print(f"Processing timestamp  {timestamp}")     
        # (Your code for calculating dates)
        schedule = nyse.schedule(start_date=timestamp.date(), end_date=timestamp.date() + pd.Timedelta(days=5))
        next_trading_day = schedule.index[1].date()
        entry_date_str = timestamp.strftime('%Y-%m-%d')
        expiry_date_str = next_trading_day.strftime('%Y-%m-%d')
        #print(f"Got entry date as {entry_date_str} and exit as {expiry_date_str}")

        # The 'try' block starts here
        try:


            utc_timestamp_for_query = timestamp.tz_localize('America/New_York').tz_convert('UTC')
            utc_timestamp_str = utc_timestamp_for_query.strftime('%Y-%m-%d %H:%M:%S')
            #print(f"Searching UTC timestamp as {utc_timestamp_str}")
            
            options_data = con.execute(f"""
                SELECT *
                FROM read_parquet('s3://duckdata/ORATS/Options/ticker=SPY/day={timestamp.strftime('%Y-%m-%d')}/*.parquet')
                WHERE 
                    ts = '{utc_timestamp_str}'  AND
                    optionType = 1 AND
                    expiry BETWEEN '{timestamp.strftime('%Y-%m-%d')}' AND '{expiry_date_str}' AND
                    strike>stockPrice*1.002 AND
                    bidPrice<1.5 AND bidPrice>0.9
                ORDER BY abs(strike - stockPrice) DESC
                LIMIT 5;
            """).df()       

            if len(options_data) > 0:
                print(f"Found data of options as {options_data}")
                target_option = options_data.iloc[0] # Select ATM+2 strike

                asset_price_at_entry = target_option['stockPrice']
                option_multiplier = 100
                delta_of_option = 0.3
                delta_per_contract = delta_of_option * option_multiplier
                

                print(f"Exposure is : {current_exposure}")
                notional_to_hedge = current_exposure * 0.5
                notional_per_contract = asset_price_at_entry * option_multiplier
                                # This is the number of shares your portfolio is equivalent to
                shares_to_hedge = notional_to_hedge / asset_price_at_entry
                qty_to_sell = 0 
                if delta_per_contract > 0:
                    qty_to_sell = int(round(shares_to_hedge / delta_per_contract))
                    print(f"Qty to sell is {qty_to_sell}")
                
                
                if qty_to_sell > 0:
                    new_position = {
                        'entry_ts': timestamp, 'strike': target_option['strike'], 'expiry': expiry_date_str,
                        'entry_price': target_option['bidPrice'], 'quantity': qty_to_sell, 'initial_delta': delta_of_option
                    }
                    open_positions.append(new_position)
                    print(f"  ✅ {timestamp.date()}: SCALED IN by selling {qty_to_sell} contracts of strike {new_position['strike']}")
            else:
                print(f"  ⚠️ {timestamp.date()}: No options data found for this entry signal.")
        except Exception as e:
            print(f"  ❌ Error processing {timestamp}: {e}")

# --- 3. Close any remaining positions at the end of the backtest ---
for pos in open_positions:
    pos['exit_reason'] = 'End of Backtest'
    trade_log.append(pos)

print("\n✅ Backtest finished.")


🚀 Starting backtest with dynamic quantity...
Processing timestamp  2024-01-04 11:30:00
Found data of options as       expiry  strike  dte  optionType                  ts    low   high  \
0 2024-01-05   472.0    2           1 2024-01-04 16:30:00  1.065  1.065   

    open  close  volume  ...  askSize  bidPrice  askPrice     bidIv     askIv  \
0  1.065  1.065   565.0  ...    192.0      1.06      1.07  0.146475  0.147454   

         iv       oi  stockPrice        day ticker  
0  0.146965  13025.0      470.74 2024-01-04    SPY  

[1 rows x 21 columns]
Exposure is : 99947.34
Qty to sell is 4
  ✅ 2024-01-04: SCALED IN by selling 4 contracts of strike 472.0
   Expiring position with strike 472.0 on 2024-01-05
   Found 3 prices in closing window. Avg exit price: $0.01
Pnl of hedge was 420.0
Processing timestamp  2024-01-05 10:15:00
Found data of options as       expiry  strike  dte  optionType                  ts    low   high  \
0 2024-01-08   471.0    4           1 2024-01-05 15:15:00  1.06

In [26]:
tl = pd.DataFrame(trade_log)

In [33]:
options_data

,expiry,strike,dte,optionType,ts,low,high,open,close,volume,...,askSize,bidPrice,askPrice,bidIv,askIv,iv,oi,stockPrice,day,ticker
0,2024-09-06,558.0,2,1,2024-09-05 14:30:00,1.175,1.175,1.175,1.175,382.0,...,188.0,1.17,1.18,0.230871,0.231751,0.231311,5071.0,552.89,2024-09-05,SPY
1,2024-09-06,557.5,2,1,2024-09-05 14:30:00,1.340,1.340,1.340,1.340,233.0,...,257.0,1.33,1.35,0.233133,0.234900,0.234016,1575.0,552.89,2024-09-05,SPY


In [29]:
tl['pnl'].dropna().cumsum()

0       420.000000
1     -1344.000000
2       616.000000
3     -3753.000000
4     -7206.333333
          ...     
56   -24057.000000
57   -23676.000000
58   -22164.000000
59   -20916.000000
60   -18828.000000
Name: pnl, Length: 61, dtype: float64

In [36]:
import pandas as pd

def find_straddle_for_day(con, ticker, timestamp, expiry_str, percentage_away):
    """
    Finds the closest matching call and put options for a straddle for an entire day.

    Args:
        con: The DuckDB connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        timestamp (pd.Timestamp): The reference timestamp to determine the stock price.
        expiry_str (str): The expiry date for the options, in 'YYYY-MM-DD' format.
        percentage_away (float): The percentage to calculate strike distance (e.g., 0.01 for 1%).

    Returns:
        pd.DataFrame: A DataFrame with the full day's data for the two selected options.
    """
    date_str = timestamp.strftime('%Y-%m-%d')
    utc_timestamp_str = timestamp.tz_localize('America/New_York').tz_convert('UTC').strftime('%Y-%m-%d %H:%M:%S')

    print(f"Finding straddle for {ticker} on {date_str} with {percentage_away:.2%} offset...")

    try:
        query = f"""
            WITH TargetInfo AS (
                SELECT 
                    stockPrice AS reference_price,
                    stockPrice * (1 + {percentage_away}) AS target_call_strike,
                    stockPrice * (1 - {percentage_away}) AS target_put_strike
                FROM read_parquet('s3://duckdata/ORATS/Options/ticker={ticker}/day={date_str}/*.parquet')
                WHERE ts = '{utc_timestamp_str}'
                LIMIT 1
            )
            SELECT 
                ts, strike, expiry, dte, optionType, volume, oi,
                bidPrice, askPrice, bidIv, askIv, iv, stockPrice, ticker
            FROM read_parquet('s3://duckdata/ORATS/Options/ticker={ticker}/day={date_str}/*.parquet')
            WHERE
                expiry = '{expiry_str}'
            QUALIFY
                ROW_NUMBER() OVER (
                    PARTITION BY optionType 
                    ORDER BY 
                        ABS(strike - CASE 
                                        WHEN optionType = 1 THEN (SELECT target_call_strike FROM TargetInfo)
                                        ELSE (SELECT target_put_strike FROM TargetInfo)
                                     END)
                ) = 1
            ORDER BY ts, optionType;
        """
        
        straddle_df = con.execute(query).df()
        
        if straddle_df.empty:
            print("INFO: Could not find matching straddle options.")
        
        return straddle_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# --- Example Usage ---

# Assume 'con' is your active DuckDB connection



In [38]:
# 1. Define the parameters
target_ticker = 'SPY'
reference_timestamp = pd.to_datetime('2024-09-06 09:30:00') # The time to check the stock price
target_expiry = '2024-09-06'
percentage_offset = 0.005 # 0.5% away from the stock price

straddle_data_df = find_straddle_for_day(
    con=con,
    ticker=target_ticker,
    timestamp=reference_timestamp,
    expiry_str=target_expiry,
    percentage_away=percentage_offset
)

if not straddle_data_df.empty:
    print("\n--- Found Straddle Data ---")
    # You will get all the day's data for two strikes: one call and one put
    print(straddle_data_df)
    print("\nSelected Strikes:")
    print(straddle_data_df['strike'].unique())

Finding straddle for SPY on 2024-09-06 with 0.50% offset...

--- Found Straddle Data ---
                   ts  strike     expiry  dte  optionType  volume      oi  \
0 2024-09-06 18:21:00   547.0 2024-09-06    1          -1    87.0  8542.0   
1 2024-09-06 19:51:00   552.5 2024-09-06    1           1     1.0  5042.0   

   bidPrice  askPrice  bidIv     askIv        iv  stockPrice ticker  
0      6.83      6.92    0.0  0.525913  0.525913      540.12    SPY  
1      0.00      0.01    0.0  0.749761  0.749761      540.39    SPY  

Selected Strikes:
[547.  552.5]


In [54]:
import pandas as pd

def find_straddle_at_timestamp(con, ticker, timestamp, expiry_str, underlying_price, percentage_away):
    """
    Finds the closest matching call and put for a straddle at a single timestamp.

    Args:
        con: The DuckDB connection object.
        ticker (str): The stock ticker (e.g., 'SPY').
        timestamp (pd.Timestamp): The specific timestamp to query (e.g., in EST).
        expiry_str (str): The expiry date for the options, in 'YYYY-MM-DD' format.
        underlying_price (float): The reference stock price to calculate strikes from.
        percentage_away (float): The percentage to calculate strike distance.

    Returns:
        pd.DataFrame: A DataFrame with the data for the two selected options,
                      or an empty DataFrame if not found.
    """
    # date_str = timestamp.strftime('%Y-%m-%d')
    # utc_timestamp_str = timestamp.tz_localize('America/New_York').tz_convert('UTC').strftime('%Y-%m-%d %H:%M:%S')
    
    # 1. Check if the input timestamp is naive (has no timezone info)
    if timestamp.tzinfo is None or timestamp.tzinfo.utcoffset(timestamp) is None:
        # If it's naive, localize it to New York time
        timestamp = timestamp.tz_localize('America/New_York')
    
    # 2. Now that we're sure it's timezone-aware, we can safely convert to UTC for the query
    date_str = timestamp.strftime('%Y-%m-%d')
    utc_timestamp_str = timestamp.tz_convert('UTC').strftime('%Y-%m-%d %H:%M:%S')
    

    print(f"Finding straddle for {ticker} at {timestamp} with {percentage_away:.2%} offset...")

    try:
        # Calculate target strikes directly in Python
        target_call_strike = underlying_price * (1 + percentage_away)
        target_put_strike = underlying_price * (1 - percentage_away)

        # The query is now faster with the added 'ts' filter
        query = f"""
            SELECT 
                ts, strike, expiry, dte, optionType, volume, oi,
                bidPrice, askPrice, bidIv, askIv, iv, stockPrice, ticker
            FROM read_parquet('s3://duckdata/ORATS/Options/ticker={ticker}/day={date_str}/*.parquet')
            WHERE
                ts = '{utc_timestamp_str}' AND -- <-- KEY CHANGE: Filter for the exact timestamp
                expiry = '{expiry_str}'
            QUALIFY
                ROW_NUMBER() OVER (
                    PARTITION BY optionType 
                    ORDER BY 
                        ABS(strike - CASE 
                                        WHEN optionType = 1 THEN {target_call_strike}
                                        ELSE {target_put_strike}
                                     END)
                ) = 1;
        """
        
        straddle_df = con.execute(query).df()
        
        if straddle_df.empty:
            print("INFO: Could not find matching straddle options at this timestamp.")
        
        straddle_df['ts'] = pd.to_datetime(straddle_df['ts'].dt.tz_localize('UTC').dt.tz_convert('America/New_York').dt.tz_localize(None))

        return straddle_df

    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()



# --- Example Usage ---

# Assume 'con' is your active DuckDB connection
# 1. Define the parameters
target_ticker = 'SPY'
target_timestamp = pd.to_datetime('2024-09-09 09:30:00') # Specific time
target_expiry = '2024-09-09'
reference_price = 540.15 
percentage_offset = 0.005 

# 2. Call the function
straddle_data_df = find_straddle_at_timestamp(
    con=con,
    ticker=target_ticker,
    timestamp=target_timestamp,
    expiry_str=target_expiry,
    underlying_price=reference_price,
    percentage_away=percentage_offset
)

if not straddle_data_df.empty:
    print("\n--- Found Straddle Data at Timestamp ---")
    print(straddle_data_df)



Finding straddle for SPY at 2024-09-09 09:30:00-04:00 with 0.50% offset...

--- Found Straddle Data at Timestamp ---
                   ts  strike     expiry  dte  optionType   volume      oi  \
0 2024-09-09 09:30:00   537.0 2024-09-09    1          -1 -32163.0  6159.0   
1 2024-09-09 09:30:00   543.0 2024-09-09    1           1 -36912.0  5349.0   

   bidPrice  askPrice     bidIv     askIv        iv  stockPrice ticker  
0      1.39      1.41  0.000000  0.254944  0.254944      544.68    SPY  
1      1.26      1.28  0.015941  0.019643  0.017792      544.68    SPY  


In [ ]:
import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
import duckdb

# ==============================================================================
# 0. SETUP: Create Sample Data and a Mock Database
# ==============================================================================


# ==============================================================================
# BACKTESTING LOGIC
# ==============================================================================

# --- Essential Change: Use a list to track multiple position layers ---
open_positions = []
trade_log = []
option_multiplier = 100
hedge_ratio = 0.50

nyse = mcal.get_calendar('NYSE')
print("\n🚀 Starting backtest with dynamic quantity...")

# Loop through each row of your exposure data
for timestamp, row in rdf.iterrows():
    marker = row['change_posn']
    current_exposure = row['net_exposure']
    change_in_exposure = row['change_in_exposure']

    # --- 1. HANDLE EXPIRATIONS FIRST ---
    positions_to_keep = []
    if open_positions:
        for position in open_positions:
            expiry_date = pd.to_datetime(position['expiry']).date()
            if timestamp.date() >= expiry_date:
                print(f"   Expiring position with strike {position['strike']} on {timestamp.date()}")
                exit_price = 0.05 # Simulate fetching a closing price
                pnl = (position['entry_price'] - exit_price) * position['quantity'] * option_multiplier
                position.update({'exit_ts': timestamp, 'exit_price': exit_price, 'pnl': pnl, 'exit_reason': 'Expired'})
                trade_log.append(position)
            else:
                positions_to_keep.append(position)
        open_positions = positions_to_keep

    # --- 2. HANDLE SIGNALS (SCALE OUT or SCALE IN) ---
    if marker == -1:
        if not open_positions:
            print(f"  ⚠️ {timestamp.date()}: Received SCALE_OUT signal, but no open positions.")
            continue
        
        decline_in_exposure = abs(change_in_exposure)
        asset_price_at_event = row['net_exposure'] / (row['net_exposure'] / 475) # Estimate price
        
        # Use parameters of the oldest position (FIFO) to calculate contracts to close
        oldest_pos = open_positions[0]
        delta_per_contract = oldest_pos['initial_delta'] * option_multiplier

        shares_to_unwind = decline_in_exposure * hedge_ratio / asset_price_at_event
        contracts_to_close = int(round(shares_to_unwind / delta_per_contract))

        if contracts_to_close > 0:
            print(f"  🔽 {timestamp.date()}: SCALING OUT by closing {contracts_to_close} contracts...")
            remaining_to_close = contracts_to_close
            temp_open_positions = []
            for position in open_positions:
                if remaining_to_close <= 0:
                    temp_open_positions.append(position)
                    continue
                
                closable_qty = min(position['quantity'], remaining_to_close)
                # In a real backtest, you would log the partial close as a trade
                position['quantity'] -= closable_qty
                remaining_to_close -= closable_qty
                
                if position['quantity'] > 0:
                    temp_open_positions.append(position)
            
            open_positions = temp_open_positions

    elif marker == 1:
        # This logic now handles both initial entry and scaling in
        increase_in_exposure = change_in_exposure
        if increase_in_exposure <= 0:
            continue

        try:
            # --- Your original data fetching logic ---
            schedule = nyse.schedule(start_date=timestamp.date(), end_date=timestamp.date() + pd.Timedelta(days=5))
            # Ensure schedule is not empty and has a second day
            if len(schedule) > 1:
                next_trading_day = schedule.index[1].date()
            else: # Handle holidays or weekends
                next_trading_day = nyse.schedule(start_date=timestamp.date() + pd.Timedelta(days=1), end_date=timestamp.date() + pd.Timedelta(days=7)).index[0].date()
            
            
            
            utc_timestamp_for_query = timestamp.tz_localize('America/New_York').tz_convert('UTC')
            utc_timestamp_str = utc_timestamp_for_query.strftime('%Y-%m-%d %H:%M:%S')
            #print(f"Searching UTC timestamp as {utc_timestamp_str}")
            
            options_data = con.execute(f"""
                SELECT *
                FROM read_parquet('s3://duckdata/ORATS/Options/ticker=SPY/day={timestamp.strftime('%Y-%m-%d')}/*.parquet')
                WHERE 
                    ts = '{utc_timestamp_str}'  AND
                    optionType = 1 AND
                    expiry BETWEEN '{timestamp.strftime('%Y-%m-%d')}' AND '{expiry_date_str}' AND
                    strike>stockPrice*1.001 AND
                    bidPrice>1
                ORDER BY abs(strike - stockPrice) DESC
                LIMIT 5;
            """).df()       

           

            if not options_data.empty:
                target_option = options_data.iloc[0]
                
                asset_price_at_entry = target_option['stockPrice']
                delta_of_option = 0.45
                delta_per_contract = delta_of_option * option_multiplier

                # Correctly hedge the INCREASE in exposure
                notional_to_hedge = increase_in_exposure * hedge_ratio
                shares_to_hedge = notional_to_hedge / asset_price_at_entry
                
                qty_to_sell = 0
                if delta_per_contract > 0:
                    qty_to_sell = int(round(shares_to_hedge / delta_per_contract))

                if qty_to_sell > 0:
                    new_position = {
                        'entry_ts': timestamp, 'strike': target_option['strike'], 'expiry': expiry_date_str,
                        'entry_price': target_option['bidPrice'], 'quantity': qty_to_sell, 'initial_delta': delta_of_option
                    }
                    open_positions.append(new_position)
                    print(f"  ✅ {timestamp.date()}: SCALED IN by selling {qty_to_sell} contracts of strike {new_position['strike']}")
            else:
                print(f"  ⚠️ {timestamp.date()}: No options data found for this entry signal.")
        except Exception as e:
            print(f"  ❌ Error processing {timestamp}: {e}")

# --- 3. Close any remaining positions at the end of the backtest ---
for pos in open_positions:
    pos['exit_reason'] = 'End of Backtest'
    trade_log.append(pos)

print("\n✅ Backtest finished.")

# ==============================================================================
# 4. REVIEW RESULTS
# ==============================================================================
print("\n--- FINAL TRADE LOG ---")
if trade_log:
    log_df = pd.DataFrame(trade_log)
    print(log_df[['entry_ts', 'strike', 'quantity', 'entry_price', 'exit_ts', 'exit_price', 'pnl', 'exit_reason']].round(2))
else:
    print("No trades were executed.")